### Bengali Grapheme Classification
This notebook is the first look at the dataset 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [2]:
train_df = pd.read_csv('data/train.csv')
class_map_df = pd.read_csv('data/class_map.csv')

In [3]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [4]:
class_map_df.head()

,component_type,label,component
0,grapheme_root,0,ং
1,grapheme_root,1,ঃ
2,grapheme_root,2,অ
3,grapheme_root,3,আ
4,grapheme_root,4,ই


In [5]:
def component_filter(df,component):
    filt = df.component_type==component
    return df[filt]

In [6]:
class_map_df.component_type.value_counts()

grapheme_root          168
vowel_diacritic         11
consonant_diacritic      7
Name: component_type, dtype: int64

In [7]:
grapheme_roots_df = component_filter(class_map_df,'grapheme_root')
vowel_diacritics_df = component_filter(class_map_df,'vowel_diacritic')
consonant_diacritics_df = component_filter(class_map_df,'consonant_diacritic')

In [8]:
def label_filter(df_filtered,label):
    filt = df_filtered.label == label
    return df_filtered[filt]

In [9]:
grapheme_roots_df.iloc[15,:]

component_type    grapheme_root
label                        15
component                   ক্ট
Name: 15, dtype: object

In [10]:
vowel_diacritics_df.iloc[9,:]

component_type    vowel_diacritic
label                           9
component                       ো
Name: 177, dtype: object

In [11]:
consonant_diacritics_df.iloc[5,:]

component_type    consonant_diacritic
label                               5
component                          ্র
Name: 184, dtype: object

In [12]:
train_image_df = pd.read_parquet('data/train_image_data_0.parquet',engine='pyarrow')

In [ ]:
train_image_head=train_image_df.head(10)

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=5,figsize=(10,8))
count=0
for row in ax:
    for col in row:
        image_to_be_shown=train_image_head.iloc[count,1:].values.reshape(137,236).astype(np.float32)
        col.imshow(image_to_be_shown)
        col.axis('off')
        count+=1
plt.tight_layout()

plt.show()

In [ ]:
#aim for today
# - train a classifier to classify grapheme roots

In [13]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [14]:
import tensorflow as tf

In [ ]:
grapheme_roots_df.shape

In [ ]:
grapheme_roots_df

In [15]:
reset_graph()

In [16]:
HEIGHT = 137
WIDTH = 236
CHANNELS=1

n_inputs = HEIGHT*WIDTH

conv1_fmaps = 32
conv1_ksize = 20
conv1_stride = 3
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 20
conv2_stride = 3
conv2_pad = "SAME"
conv2_dropout_rate = 0.25

pool3_maps = conv2_fmaps

n_fc1 = 256
fc1_dropout_rate = 0.5

n_outputs = 11

In [ ]:
train_image_df.shape

In [17]:
X_train = train_image_df.iloc[:10000,1:].values.astype(np.float32)

In [ ]:
X_train.shape

In [18]:
X_test = train_image_df.iloc[10000:15000,1:].values.astype(np.float32)

In [ ]:
X_test.shape

In [26]:
y_train = train_df.iloc[:10000,2].values.astype(np.int32)

In [ ]:
y_train.shape

In [27]:
y_test = train_df.iloc[10000:15000,2].values.astype(np.int32)

In [ ]:
y_test.shape

In [21]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ohe = OneHotEncoder(sparse=False)

In [28]:
y_train_ohe=ohe.fit_transform(y_train.reshape(-1,1))
y_test_ohe=ohe.fit_transform(y_test.reshape(-1,1))

In [29]:
y_train_ohe.shape, y_test_ohe.shape

((10000, 11), (5000, 11))

In [ ]:
reset_graph()

In [30]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32,shape=[None,n_inputs],name="X")
    X_reshaped = tf.reshape(X,shape=[-1,HEIGHT,WIDTH,CHANNELS])
    y = tf.placeholder(tf.int32, shape=[None],name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

In [31]:
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps,kernel_size = conv1_ksize, strides = conv1_stride,
                         padding=conv1_pad,activation=tf.nn.relu,name="conv1")

conv2 = tf.layers.conv2d(conv1,filters=conv2_fmaps, kernel_size = conv2_ksize, strides = conv2_stride,
                        padding=conv2_pad, activation=tf.nn.relu, name="conv2")

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [32]:
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2,ksize=[1,10,10,1], strides=[1,2,2,1],padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1,pool3_maps*1800])
    pool3_flat_drop = tf.layers.dropout(pool3_flat,conv2_dropout_rate, training=training)

Instructions for updating:
Use keras.layers.dropout instead.


In [33]:
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf. layers.dropout(fc1,fc1_dropout_rate,training=training)

Instructions for updating:
Use keras.layers.Dense instead.


In [34]:
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1_drop,n_outputs,name="output")
    Y_proba = tf.nn.softmax(logits,name="Y_proba")

In [35]:
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [36]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [37]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [38]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [39]:
n_epochs = 10
batch_size = 50
iteration = 0

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            iteration += 1
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: X_valid, y: y_valid})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch {}, last batch accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_batch * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./grapheme_roots_model")

InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [1,11] and labels shape [50]
	 [[node train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits':
  File "/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/

In [ ]:
#trying keras instead

In [52]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input

In [53]:
from keras.optimizers import Adam

In [54]:
inputs = Input(shape=(137,236,1))

In [55]:
model = Conv2D(filters=64,kernel_size=(10,10),padding="SAME",activation="relu",input_shape=(137,236,1))(inputs)

In [56]:
model = Conv2D(filters=64,kernel_size=(10,10),padding="SAME",activation="relu",input_shape=(137,236,1))(model)

In [57]:
model = Dropout(rate=0.25)(model)

In [58]:
model = MaxPool2D(pool_size=(20,20))(model)

In [59]:
model = Dropout(rate=0.25)(model)

In [60]:
model =Flatten()(model)

In [61]:
model = Dense(264,activation="relu")(model)

In [62]:
model = Dropout(rate=0.5)(model)

In [63]:
head_root = Dense(11,activation="softmax")(model)

In [64]:
model = Model(inputs=inputs,outputs=head_root)

In [65]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 137, 236, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 137, 236, 64)      6464      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 137, 236, 64)      409664    
_________________________________________________________________
dropout_4 (Dropout)          (None, 137, 236, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 11, 64)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 11, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4224)              0   

In [66]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [67]:
batch_size=256
epochs = 30

In [71]:
X_train= X_train.reshape(-1,137,236,1)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False)

In [ ]:
y_train_ohe=ohe.fit_transform(y_train.reshape(-1,1))

In [ ]:
y_train_ohe.shape

In [ ]:
y_test.shape

In [72]:
X_test = X_test.reshape(-1,137,236,1)

In [ ]:
X_test.shape

In [69]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [75]:
history = model.fit(x=X_train,y=y_train_ohe,batch_size=50,epochs=epochs,validation_data=(X_test,y_test_ohe))




Train on 10000 samples, validate on 5000 samples
Epoch 1/30
10000/10000 [==============================] - 76s 8ms/step - loss: 4.5898 - accuracy: 0.1839 - val_loss: 2.2369 - val_accuracy: 0.2084
Epoch 2/30
10000/10000 [==============================] - 72s 7ms/step - loss: 2.1604 - accuracy: 0.2022 - val_loss: 2.1412 - val_accuracy: 0.2084
Epoch 3/30
10000/10000 [==============================] - 73s 7ms/step - loss: 2.1351 - accuracy: 0.1973 - val_loss: 2.1382 - val_accuracy: 0.2084
Epoch 4/30
10000/10000 [==============================] - 73s 7ms/step - loss: 2.1336 - accuracy: 0.1999 - val_loss: 2.1382 - val_accuracy: 0.2084
Epoch 5/30
 8550/10000 [========================>.....] - ETA: 9s - loss: 2.1337 - accuracy: 0.1988

KeyboardInterrupt: 